In [2]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
old_data_loc = '/Users/jonaheaton/ReviveMed Dropbox/Jonah Eaton/development_CohortCombination/alignment_RCC_2024_Feb_27/April_05_Data'

new_data_loc = '/Users/jonaheaton/ReviveMed Dropbox/Jonah Eaton/development_CohortCombination/alignment_RCC_2024_Feb_27/May_28_Data'


In [4]:
metadata_file = f'{old_data_loc}/metadata.csv'
# Read the metadata file
metadata = pd.read_csv(metadata_file, index_col=0)

In [5]:
metadata

,cohort_id,Study ID,Cohort Label,Cohort ID,OS,OS_Event,Age,subject ID,study_week,Region,...,is Pediatric,Cohort Label ENC,Study ID ENC,file id,Age Range (min),Age Range (max),IMDC,IMDC ORDINAL,IMDC BINARY,is Female
0582_Marios_PD1_Inhib2_HP-EA003798-7.mzML,541,ST001237,adult_cancer,541,50.562628,0.0,62.0,CA209025-111-657,baseline,REST OF WORLD,...,False,0,4,0582_Marios_PD1_Inhib2_HP-EA003798-7.mzML,62.0,62.0,FAVORABLE,2.0,1.0,1.0
0293_Marios_PD1_Inhib2_HP-E9101785-7.mzML,541,ST001237,adult_cancer,541,24.607803,1.0,59.0,CA209025-33-12,baseline,WESTERN EUROPE,...,False,0,4,0293_Marios_PD1_Inhib2_HP-E9101785-7.mzML,59.0,59.0,INTERMEDIATE,1.0,NaN,0.0
0219_Marios_PD1_Inhib2_HP-EA003799-7.mzML,541,ST001237,adult_cancer,541,52.073922,0.0,66.0,CA209025-111-778,baseline,REST OF WORLD,...,False,0,4,0219_Marios_PD1_Inhib2_HP-EA003799-7.mzML,66.0,66.0,INTERMEDIATE,1.0,NaN,0.0
0962_Marios_PD1_Inhib2_HP-E9784758-7.mzML,541,ST001237,adult_cancer,541,24.246407,1.0,80.0,CA209025-180-1001,baseline,REST OF WORLD,...,False,0,4,0962_Marios_PD1_Inhib2_HP-E9784758-7.mzML,80.0,80.0,POOR,0.0,0.0,1.0
0218_Marios_PD1_Inhib2_HP-E9123067-7.mzML,541,ST001237,adult_cancer,541,3.614000,1.0,60.0,CA209025-113-50,baseline,WESTERN EUROPE,...,False,0,4,0218_Marios_PD1_Inhib2_HP-E9123067-7.mzML,60.0,60.0,NOT REPORTED,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1188_Marios_PD1_Inhib2_HP-AE052720-7.mzML,541,ST001237,adult_cancer,541,60.254620,0.0,56.0,CA209025-139-227,baseline,US/CANADA,...,False,0,4,1188_Marios_PD1_Inhib2_HP-AE052720-7.mzML,56.0,56.0,INTERMEDIATE,1.0,NaN,0.0
1023_Marios_PD1_Inhib2_HP-AD185798-7.mzML,541,ST001237,adult_cancer,541,28.977413,1.0,72.0,CA209025-1-27,baseline,US/CANADA,...,False,0,4,1023_Marios_PD1_Inhib2_HP-AD185798-7.mzML,72.0,72.0,FAVORABLE,2.0,1.0,0.0
1093_Marios_PD1_Inhib2_HP-AE726342-7.mzML,541,ST001237,adult_cancer,541,25.856300,1.0,58.0,CA209025-138-872,baseline,US/CANADA,...,False,0,4,1093_Marios_PD1_Inhib2_HP-AE726342-7.mzML,58.0,58.0,FAVORABLE,2.0,1.0,1.0
0925_Marios_PD1_Inhib2_HP-AE673799-7.mzML,541,ST001237,adult_cancer,541,19.679700,1.0,52.0,CA209025-144-717,baseline,US/CANADA,...,False,0,4,0925_Marios_PD1_Inhib2_HP-AE673799-7.mzML,52.0,52.0,FAVORABLE,2.0,1.0,0.0


In [90]:
metadata['Study ID'].value_counts()

Study ID
ST001932    4482
ST001422    2051
ST001931    2044
ST001428    1522
ST001237    1379
ST002331    1315
ST001423    1192
ST000909     742
ST001849     691
ST002027     356
ST001408     349
ST002112     335
ST001236     271
ST001918     271
ST002251     242
ST001519     166
ST002244     122
ST000388      95
ST000422      60
Name: count, dtype: int64

## ST000422
no useful metadata

### Correct the cohort label for ST000422
previously the cohort label for ST000422 was set to adult_cancer, but it should be adult_other

In [11]:
metadata.groupby('Cohort Label')['Cohort Label ENC'].mean()

Cohort Label
adult_cancer       0.0
adult_other        1.0
pediatric_CMD      2.0
pediatric_other    3.0
Name: Cohort Label ENC, dtype: float64

In [6]:
print(metadata[metadata['Study ID'] == 'ST000422']['Cohort Label'].value_counts())

Cohort Label
adult_cancer    60
Name: count, dtype: int64


In [10]:
metadata.loc[metadata['Study ID'] == 'ST000422','Cohort Label'] = 'adult_other'
metadata.loc[metadata['Study ID'] == 'ST000422','Cohort Label ENC'] = 1

Study ID
ST001932    4482
ST001422    2051
ST001931    2044
ST001428    1522
ST001237    1379
ST002331    1315
ST001423    1192
ST000909     742
ST001849     691
ST002027     356
ST001408     349
ST002112     335
ST001236     271
ST001918     271
ST002251     242
ST001519     166
ST002244     122
ST000388      95
ST000422      60
Name: count, dtype: int64

### Leila Note (Friday May 24th):
So I realized we don't need to know baseline and 3 years for cancer labeling
Because if a patient has "-" value in the cancer detection columns, its label would be non-cancer
and if a patient has "1" in in the cancer detection columns, its label would be "cancer", even the cancer diagnosed later, the earlier time point is very interesting, as it reflect cancer risk

## ST001422, ST001423
- lots of patients have gender metadata
- many have Age, BMI and cancer status


In [45]:
asprin_metadata_file = '/Users/jonaheaton/ReviveMed Dropbox/Jonah Eaton/mzLearn_preTraining/source_metadata/Aspirin_PR000730/combined_patient_samples.xlsx'

asprin_metadata = pd.read_excel(asprin_metadata_file,sheet_name='combined_patient_samples (2)', index_col=0, header=0)
asprin_metadata = asprin_metadata[~(asprin_metadata['IonizationMode'] == 'C18neg')].copy()

### Compare to paper

2020_Metabolomics Analysis of Aspirin’s Effects in Human Colon Tissue and Associations with Adenoma Risk
In table 1 of the supplements

Total number of patients considered: 100+114+111 = 325 patients

"≥1 Adenoma at Year 3 Colonoscopy" has 33+39+47 = 119 patients

"≥1 Advanced Adenoma at Year 3 Colonoscopy" has 6+8+13 = 27 patients

"High-risk Findings at Year 3 Colonoscopy" has 9+11+16= 36 patients

In [81]:
asprin_patients = asprin_metadata.groupby('PatientID').first()
asprin_patients = asprin_patients[~asprin_patients['BMI'].isna()].copy()
patient_total = asprin_patients.shape[0]
print(f'Number of patients in aspirin study with BMI: {patient_total}')


Tradyn_total = asprin_patients['Tradyn'].value_counts()[1]
print(f'Number of patients in aspirin study with Tradyn: {Tradyn_total}')

Advtradyn_total = asprin_patients['Advtradyn'].value_counts()[1]
print(f'Number of patients in aspirin study with Advtradyn: {Advtradyn_total}')

Hradvtradyn_total = asprin_patients['Hradvtradyn'].value_counts()[1]
print(f'Number of patients in aspirin study with Hradvtradyn: {Hradvtradyn_total}')

Number of patients in aspirin study with BMI: 325
Number of patients in aspirin study with Tradyn: 119
Number of patients in aspirin study with Advtradyn: 27
Number of patients in aspirin study with Hradvtradyn: 36


In [48]:
subset_ST001423 = asprin_metadata[~asprin_metadata['RAW_FILE_NAME_ST001423'].isna()].copy()
subset_ST001423.index = subset_ST001423['RAW_FILE_NAME_ST001423'].apply(lambda x: x+'.mzML')

subset_ST001422 = asprin_metadata[~asprin_metadata['RAW_FILE_NAME_ST001422'].isna()].copy()
subset_ST001422.index = subset_ST001422['RAW_FILE_NAME_ST001422'].apply(lambda x: x+'.mzML')


In [52]:
missing_files_ST001423 = subset_ST001423.index.difference(metadata.index)
missing_files_ST001422 = subset_ST001422.index.difference(metadata.index)

print(f'Number of missing files in ST001423: {len(missing_files_ST001423)}')
print(f'Number of missing files in ST001422: {len(missing_files_ST001422)}')

Number of missing files in ST001423: 86
Number of missing files in ST001422: 8


In [89]:
print('Missing with Sex info', np.sum(~subset_ST001423.loc[missing_files_ST001423,'Sex'].isna()))
print('Missing with Age info', np.sum(~subset_ST001423.loc[missing_files_ST001423,'Age'].isna()))

print('Missing with Sex info', np.sum(~subset_ST001422.loc[missing_files_ST001422,'Sex'].isna()))
print('Missing with Age info', np.sum(~subset_ST001422.loc[missing_files_ST001422,'Age'].isna()))



Missing with Sex info 86
Missing with Age info 46
Missing with Sex info 8
Missing with Age info 2


#### Note 

<span style="color:blue">I think I was too quick to remove outliers from this dataset. 
between these two studies, there are another 94 samples that have Sex
with 48 of those having Age, BMI and cancer information, </span>



In [76]:
Patient_total = (subset_ST001423[~subset_ST001423['BMI'].isna()].shape[0] + subset_ST001422[~subset_ST001422['BMI'].isna()].shape[0])/2
print(f'Patient total: {Patient_total}')

Tradyn_total = (subset_ST001423['Tradyn'].value_counts()[1] + subset_ST001422['Tradyn'].value_counts()[1])/2
print(f'Tradyn total: {Tradyn_total}') #119 

Advtradyn_total = (subset_ST001423['Advtradyn'].value_counts()[1] + subset_ST001422['Advtradyn'].value_counts()[1])/2
print(f'Advtradyn total: {Advtradyn_total}')

Hradvtradyn_total = (subset_ST001423['Hradvtradyn'].value_counts()[1] + subset_ST001422['Hradvtradyn'].value_counts()[1])/2
print(f'Hradvtradyn total: {Hradvtradyn_total}')

Patient total: 293.0
Tradyn total: 109.0
Advtradyn total: 24.0
Hradvtradyn total: 33.0


In [ ]:
# we want to add columns for has_cancer, bmi, 
# update the age, update the Sex


## ST000388
- there should already be gender
- we need to add cancer status
- we can add smoking status



## ST001408
- we need to add gender, all patients are men
- all patients have cancer

## ST001236, ST001237
- all patients have cancer


## ST002244
- all patients are non-cancer

## ST002112
- we shoud already have gender
- all patients are non-cancer

## ST002027
- add gender info, all patients are women
- all patients are non cancer

## ST001918
- add BMI
- we should already have gender info
- can optionally add smoking status
- all patients are non-cancer

## ST001849
- we should already have age and gender information
- we can add BMI
- we can optionally add smoking status
- we have some cancer labels, but since patients have covid, unclear if we can use all of them


for cancer labels
- cancer without covid : cancer label
- no-cancer w/wo covid: no-cancer label
- cancer with covid: NA label

## ST001519
- we should already have age and gender labels
- add BMI
- all patients are non-cancer

## ST001932
- all patients are non-cancer

## ST001428
- all patients are non cancer

## ST000909
- all patients are non cancer

## ST002331
- all patients are non cancer
- we should already have gender and age information

## ST002251
- all patients are non cancer
-  we should already have age and gender information
-  add BMI data

## ST001931
- all patients are non cancer